# Transformation des données de pandémies - Partie 2: Dédoublonnage et Agrégation

Ce notebook réalise la suppression des doublons et l'agrégation des données nettoyées.

## 1. Configuration initiale

Importation des bibliothèques nécessaires et chargement des données nettoyées.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

# Configuration de la visualisation
%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Affichage du chemin
print(f"Dossier des données transformées: {dossier_donnees_transformees}")

In [ ]:
# Chargement des données nettoyées
print("Chargement des données nettoyées...")

donnees = {}

try:
    # Chargement des données COVID-19 nettoyées
    fichier_covid19 = dossier_donnees_transformees / "covid19_nettoye.csv"
    if fichier_covid19.exists():
        donnees["covid19"] = pd.read_csv(fichier_covid19, parse_dates=["date"])
        print(f"Données COVID-19 nettoyées chargées: {donnees['covid19'].shape[0]} lignes, {donnees['covid19'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger les données COVID-19 nettoyées: fichier non trouvé")
    
    # Chargement des données MPOX nettoyées
    fichier_mpox = dossier_donnees_transformees / "mpox_nettoye.csv"
    if fichier_mpox.exists():
        donnees["mpox"] = pd.read_csv(fichier_mpox, parse_dates=["date"])
        print(f"Données MPOX nettoyées chargées: {donnees['mpox'].shape[0]} lignes, {donnees['mpox'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger les données MPOX nettoyées: fichier non trouvé")
    
    # Chargement des données Worldometer nettoyées
    fichier_worldometer = dossier_donnees_transformees / "worldometer_nettoye.csv"
    if fichier_worldometer.exists():
        donnees["worldometer"] = pd.read_csv(fichier_worldometer, parse_dates=["date"])
        print(f"Données Worldometer nettoyées chargées: {donnees['worldometer'].shape[0]} lignes, {donnees['worldometer'].shape[1]} colonnes")
    else:
        print(f"Impossible de charger les données Worldometer nettoyées: fichier non trouvé")
    
    print("\nChargement des données nettoyées terminé avec succès!")
except Exception as e:
    print(f"Erreur lors du chargement des données nettoyées: {e}")

## 2. Suppression des doublons dans les données COVID-19

Identification et suppression des entrées en double dans les données COVID-19.

In [ ]:
# Suppression des doublons dans les données COVID-19
if "covid19" in donnees:
    print("Suppression des doublons dans les données COVID-19...")
    
    # Affichage du nombre de lignes avant suppression des doublons
    print(f"Nombre de lignes avant suppression des doublons: {donnees['covid19'].shape[0]}")
    
    # Vérification des doublons par pays, région et date
    doublons = donnees['covid19'].duplicated(subset=["date", "pays", "region"], keep=False)
    print(f"Nombre de doublons identifiés: {doublons.sum()}")
    
    if doublons.sum() > 0:
        print("\nExemples de doublons:")
        display(donnees['covid19'][doublons].head())
    
    # Suppression des doublons en gardant la première occurrence
    donnees['covid19'] = donnees['covid19'].drop_duplicates(subset=["date", "pays", "region"], keep="first")
    
    # Affichage du nombre de lignes après suppression des doublons
    print(f"Nombre de lignes après suppression des doublons: {donnees['covid19'].shape[0]}")
    print("Suppression des doublons dans les données COVID-19 terminée avec succès!")

## 3. Suppression des doublons dans les données MPOX

Identification et suppression des entrées en double dans les données MPOX.

In [ ]:
# Suppression des doublons dans les données MPOX
if "mpox" in donnees:
    print("Suppression des doublons dans les données MPOX...")
    
    # Affichage du nombre de lignes avant suppression des doublons
    print(f"Nombre de lignes avant suppression des doublons: {donnees['mpox'].shape[0]}")
    
    # Vérification des doublons par pays et date
    doublons = donnees['mpox'].duplicated(subset=["date", "pays"], keep=False)
    print(f"Nombre de doublons identifiés: {doublons.sum()}")
    
    if doublons.sum() > 0:
        print("\nExemples de doublons:")
        display(donnees['mpox'][doublons].head())
    
    # Suppression des doublons en gardant la première occurrence
    donnees['mpox'] = donnees['mpox'].drop_duplicates(subset=["date", "pays"], keep="first")
    
    # Affichage du nombre de lignes après suppression des doublons
    print(f"Nombre de lignes après suppression des doublons: {donnees['mpox'].shape[0]}")
    print("Suppression des doublons dans les données MPOX terminée avec succès!")

## 4. Suppression des doublons dans les données Worldometer

Identification et suppression des entrées en double dans les données Worldometer.

In [ ]:
# Suppression des doublons dans les données Worldometer
if "worldometer" in donnees:
    print("Suppression des doublons dans les données Worldometer...")
    
    # Affichage du nombre de lignes avant suppression des doublons
    print(f"Nombre de lignes avant suppression des doublons: {donnees['worldometer'].shape[0]}")
    
    # Vérification des doublons par pays et date
    doublons = donnees['worldometer'].duplicated(subset=["date", "pays"], keep=False)
    print(f"Nombre de doublons identifiés: {doublons.sum()}")
    
    if doublons.sum() > 0:
        print("\nExemples de doublons:")
        display(donnees['worldometer'][doublons].head())
    
    # Suppression des doublons en gardant la première occurrence
    donnees['worldometer'] = donnees['worldometer'].drop_duplicates(subset=["date", "pays"], keep="first")
    
    # Affichage du nombre de lignes après suppression des doublons
    print(f"Nombre de lignes après suppression des doublons: {donnees['worldometer'].shape[0]}")
    print("Suppression des doublons dans les données Worldometer terminée avec succès!")

## 5. Agrégation des données COVID-19

Agrégation des données COVID-19 par pays et date pour réduire le volume de données.

In [ ]:
# Agrégation des données COVID-19
if "covid19" in donnees:
    print("Agrégation des données COVID-19...")
    
    # Affichage du nombre de lignes avant agrégation
    print(f"Nombre de lignes avant agrégation: {donnees['covid19'].shape[0]}")
    
    # Agrégation des données par pays et date
    covid19_agg = donnees['covid19'].groupby(["date", "pays", "type_pandemie"]).agg({
        "cas_confirmes": "sum",
        "deces": "sum",
        "guerisons": "sum",
        "cas_actifs": "sum"
    }).reset_index()
    
    # Affichage du nombre de lignes après agrégation
    print(f"Nombre de lignes après agrégation: {covid19_agg.shape[0]}")
    print("\nAperçu des données après agrégation:")
    display(covid19_agg.head())
    
    # Sauvegarde des données agrégées
    donnees["covid19_agg"] = covid19_agg
    print("Agrégation des données COVID-19 terminée avec succès!")

## 6. Agrégation des données Worldometer

Agrégation des données Worldometer par pays et date pour réduire le volume de données.

In [ ]:
# Agrégation des données Worldometer
if "worldometer" in donnees:
    print("Agrégation des données Worldometer...")
    
    # Affichage du nombre de lignes avant agrégation
    print(f"Nombre de lignes avant agrégation: {donnees['worldometer'].shape[0]}")
    
    # Agrégation des données par pays et date
    worldometer_agg = donnees['worldometer'].groupby(["date", "pays", "type_pandemie"]).agg({
        "cas_confirmes": "sum",
        "deces": "sum"
    }).reset_index()
    
    # Affichage du nombre de lignes après agrégation
    print(f"Nombre de lignes après agrégation: {worldometer_agg.shape[0]}")
    print("\nAperçu des données après agrégation:")
    display(worldometer_agg.head())
    
    # Sauvegarde des données agrégées
    donnees["worldometer_agg"] = worldometer_agg
    print("Agrégation des données Worldometer terminée avec succès!")

## 7. Fusion des données COVID-19

Fusion des données COVID-19 provenant des différentes sources pour obtenir un jeu de données unifié.

In [ ]:
# Fusion des données COVID-19
if "covid19_agg" in donnees and "worldometer_agg" in donnees:
    print("Fusion des données COVID-19...")
    
    # Sélection des colonnes communes
    covid19_fusion = donnees["covid19_agg"][["date", "pays", "type_pandemie", "cas_confirmes", "deces"]]
    worldometer_fusion = donnees["worldometer_agg"][["date", "pays", "type_pandemie", "cas_confirmes", "deces"]]
    
    # Ajout d'une colonne pour la source des données
    covid19_fusion["source"] = "covid19"
    worldometer_fusion["source"] = "worldometer"
    
    # Fusion des données
    covid19_fusionne = pd.concat([covid19_fusion, worldometer_fusion])
    
    # Affichage du nombre de lignes après fusion
    print(f"Nombre de lignes après fusion: {covid19_fusionne.shape[0]}")
    print("\nAperçu des données après fusion:")
    display(covid19_fusionne.head())
    
    # Vérification des doublons après fusion
    doublons = covid19_fusionne.duplicated(subset=["date", "pays"], keep=False)
    print(f"Nombre de doublons après fusion: {doublons.sum()}")
    
    # Agrégation des données fusionnées par pays, date et type de pandémie
    covid19_fusionne_agg = covid19_fusionne.groupby(["date", "pays", "type_pandemie"]).agg({
        "cas_confirmes": "max",  # On prend le maximum pour éviter les sous-estimations
        "deces": "max"
    }).reset_index()
    
    # Affichage du nombre de lignes après agrégation finale
    print(f"Nombre de lignes après agrégation finale: {covid19_fusionne_agg.shape[0]}")
    print("\nAperçu des données après agrégation finale:")
    display(covid19_fusionne_agg.head())
    
    # Sauvegarde des données fusionnées
    donnees["covid19_fusionne"] = covid19_fusionne_agg
    print("Fusion des données COVID-19 terminée avec succès!")

## 8. Création du jeu de données final

Création du jeu de données final en combinant les données COVID-19 fusionnées et les données MPOX.

In [ ]:
# Création du jeu de données final
if "covid19_fusionne" in donnees and "mpox" in donnees:
    print("Création du jeu de données final...")
    
    # Sélection des colonnes pertinentes pour les données MPOX
    mpox_final = donnees["mpox"][["date", "pays", "type_pandemie", "cas_confirmes", "deces"]]
    
    # Combinaison des données COVID-19 et MPOX
    donnees_final = pd.concat([donnees["covid19_fusionne"], mpox_final])
    
    # Affichage du nombre de lignes du jeu de données final
    print(f"Nombre de lignes du jeu de données final: {donnees_final.shape[0]}")
    print("\nAperçu du jeu de données final:")
    display(donnees_final.head())
    
    # Vérification des valeurs manquantes
    valeurs_manquantes = donnees_final.isnull().sum()
    print("\nValeurs manquantes dans le jeu de données final:")
    display(valeurs_manquantes[valeurs_manquantes > 0])
    
    # Gestion des valeurs manquantes
    donnees_final = donnees_final.fillna(0)
    
    # Conversion des types de données
    donnees_final["cas_confirmes"] = donnees_final["cas_confirmes"].astype(int)
    donnees_final["deces"] = donnees_final["deces"].astype(int)
    
    # Sauvegarde du jeu de données final
    donnees["donnees_final"] = donnees_final
    print("Création du jeu de données final terminée avec succès!")

## 9. Sauvegarde du jeu de données final

Sauvegarde du jeu de données final pour l'étape suivante de normalisation.

In [ ]:
# Sauvegarde du jeu de données final
if "donnees_final" in donnees:
    print("Sauvegarde du jeu de données final...")
    
    try:
        # Sauvegarde du jeu de données final
        chemin_fichier = dossier_donnees_transformees / "donnees_final.csv"
        donnees["donnees_final"].to_csv(chemin_fichier, index=False)
        print(f"Jeu de données final sauvegardé: {chemin_fichier}")
        print("\nSauvegarde du jeu de données final terminée avec succès!")
    except Exception as e:
        print(f"Erreur lors de la sauvegarde du jeu de données final: {e}")

## Conclusion

Dans ce notebook, nous avons réalisé la suppression des doublons et l'agrégation des données nettoyées. Les principales opérations ont été :

1. Suppression des doublons dans les données COVID-19, MPOX et Worldometer
2. Agrégation des données par pays et date pour réduire le volume de données
3. Fusion des données COVID-19 provenant des différentes sources
4. Création du jeu de données final en combinant les données COVID-19 et MPOX

Le jeu de données final a été sauvegardé dans un fichier CSV pour l'étape suivante de normalisation, qui sera réalisée dans le prochain notebook.